In [11]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON

def get_wikidata_id(name):
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'language': 'en',
        'search': name
    }
    response = requests.get(url, params=params)
    data = response.json()
    if len(data['search']) > 0:
        return data['search'][0]['id']
    else:
        return None

def get_knowledge_graph(wikidata_id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    query = """
SELECT    ?companyLabel ?ticker ?countryLabel ?headquartersLabel ?ceoLabel ?industryLabel ?businessLabel ?productLabel ?competitorLabel ?nicknameLabel ?subsidiaryLabel WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
      ?company wdt:P361 wd:%s.
      OPTIONAL {
        ?company p:P414 ?exchange . 
        ?exchange ps:P414 wd:Q13677 .
        ?exchange pq:P249 ?ticker .
      }
      OPTIONAL { ?company wdt:P17 ?country. }
      OPTIONAL { ?company wdt:P159 ?headquarters. }
      OPTIONAL { ?company wdt:P169 ?ceo. }
      OPTIONAL { ?company wdt:P452 ?industry. }
      OPTIONAL { ?company wdt:P199 ?business. }
      OPTIONAL { ?company wdt:P1056 ?product. }
#       OPTIONAL { ?company wdt:P169 ?competitor. }
      OPTIONAL { ?company wdt:P1449 ?nickname. }
      OPTIONAL { ?company wdt:P355 ?subsidiary. }
    }
    """ % (wikidata_id)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results
    
    knowledge_graph = {}
    for result in results["results"]["bindings"]:
        item_label = result['itemLabel']['value']
        property_label = result['propertyLabel']['value']
        value_label = result['valueLabel']['value']
        if property_label not in knowledge_graph:
            knowledge_graph[property_label] = []
        knowledge_graph[property_label].append(value_label)
    return knowledge_graph

In [12]:
wikidata_id = get_wikidata_id('Dow Jones Industrial Average')
# wikidata_id

In [13]:
knowledge_graph = get_knowledge_graph(wikidata_id)

In [19]:
knowledge_graph

{'head': {'vars': ['companyLabel',
   'ticker',
   'countryLabel',
   'headquartersLabel',
   'ceoLabel',
   'industryLabel',
   'businessLabel',
   'productLabel',
   'competitorLabel',
   'nicknameLabel',
   'subsidiaryLabel']},
 'results': {'bindings': [{'companyLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Apple'},
    'countryLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'United States of America'},
    'headquartersLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Cupertino'},
    'ceoLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Tim Cook'},
    'industryLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'electronics'},
    'businessLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Apple Store'},
    'productLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'iPod'},
    'subsidiaryLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Apple Store'}},
   {'companyLabel': 

In [20]:
# from rdflib import Graph
# # Create a Graph object
# graph = Graph()
# data = knowledge_graph
# # Add the triples from the JSON response to the Graph object
# for triple in data["results"]["bindings"]:
#     print(triple)
#     graph.add((triple["companyLabel"]["value"], triple["countryLabel"]["value"], triple["headquartersLabel"]["value"]))

# # Save the Graph object to a file
# graph.serialize("us_companies_current_ceo.ttl")

In [22]:
import requests
from bs4 import BeautifulSoup
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd



news_data = get_news_data('Dow Jones Index', 2022)

In [23]:
print(pd.json_normalize(news_data))

                                                title  \
0   Stocks close lower for a fourth day on renewed...   
1   Dow closes more than 250 points lower Wednesda...   
2    Dow Jones Today: Index Falls Following Rate Hike   
3   Dow tumbles more than 300 points as banking se...   
4   Nearly half of Americans are nervous about the...   
..                                                ...   
85  S&P 500 posts worst day in more than a month, ...   
86  Dow Jones Industrial Average (DJIA) Lags Marke...   
87  3 Dow Jones Industrial Average Stocks To Watch...   
88  3 Dow Jones Industrial Average Stocks With 45%...   
89  Why the Dow Crushed the Rest of the Stock Mark...   

                                                 link  
0   https://news.google.com/articles/CBMiRGh0dHBzO...  
1   https://news.google.com/articles/CBMiRGh0dHBzO...  
2   https://news.google.com/articles/CBMiP2h0dHBzO...  
3   https://news.google.com/articles/CBMiRGh0dHBzO...  
4   https://news.google.com/article

In [4]:
import requests

def get_wikidata(query):
    url = 'https://query.wikidata.org/sparql'
    r = requests.get(url, params={'query': query, 'format': 'json'})
    return r.json()

query = """
SELECT ?item ?itemLabel ?value
WHERE
{
  ?item wdt:P414 wd:Q13677.
  ?item wdt:P2927 ?value.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

data = get_wikidata(query)

for result in data['results']['bindings']:
    print(result['itemLabel']['value'], result['value']['value'])

KeyError: 'value'

In [29]:
import requests
import pandas as pd

def get_companies():
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    query = """
SELECT    ?companyLabel ?ticker ?countryLabel ?headquartersLabel ?ceoLabel ?industryLabel ?businessLabel ?productLabel ?competitorLabel ?nicknameLabel ?subsidiaryLabel WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
      ?company wdt:P361 wd: Q180816.
      OPTIONAL {
        ?company p:P414 ?exchange . 
        ?exchange ps:P414 wd:Q13677 .
        ?exchange pq:P249 ?ticker .
      }
      OPTIONAL { ?company wdt:P17 ?country. }
      OPTIONAL { ?company wdt:P159 ?headquarters. }
      OPTIONAL { ?company wdt:P169 ?ceo. }
      OPTIONAL { ?company wdt:P452 ?industry. }
      OPTIONAL { ?company wdt:P199 ?business. }
      OPTIONAL { ?company wdt:P1056 ?product. }
#       OPTIONAL { ?company wdt:P169 ?competitor. }
      OPTIONAL { ?company wdt:P1449 ?nickname. }
      OPTIONAL { ?company wdt:P355 ?subsidiary. }
    }
    """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return query
    url = 'https://query.wikidata.org/sparql'
    r = requests.get(url, params={'format': 'json', 'query': query})
    data = r.json()
    companies = []
    for item in data['results']['bindings']:
        company = {}
        company['name'] = item['companyLabel']['value']
        if 'ticker' in item:
            company['ticker'] = item['ticker']['value']
        if 'headquartersLabel' in item:
            company['headquarters'] = item['headquartersLabel']['value']
        if 'ceoLabel' in item:
            company['ceo'] = item['ceoLabel']['value']
        if 'industryLabel' in item:
            company['industry'] = item['industryLabel']['value']
        if 'businessLabel' in item:
            company['business'] = item['businessLabel']['value']
        if 'productLabel' in item:
            company['product'] = item['productLabel']['value']
#         if 'competitorLabel' in item:
#             company['competitor'] = item['competitorLabel']['value']
        if 'nicknameLabel' in item:
            company['nickname'] = item['nicknameLabel']['value']
        if 'subsidiaryLabel' in item:
            company['subsidiary'] = item['subsidiaryLabel']['value']
#         if 'revenue' in item:
#             company['revenue'] = float(item['revenue']['value'])
#         if 'net_income' in item:
#             company['net_income'] = float(item['net_income']['value'])
        companies.append(company)
    return pd.DataFrame(companies)




In [30]:
# Example usage
dow_jones_companies_df = get_companies()

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'SPARQL-QUERY: queryStr=\nSELECT    ?companyLabel ?ticker ?countryLabel ?headquartersLabel ?ceoLabel ?industryLabel ?businessLabel ?productLabel ?competitorLabel ?nicknameLabel ?subsidiaryLabel WHERE {\n      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }\n      ?company wdt:P361 wd: Q180816.\n      OPTIONAL {\n        ?company p:P414 ?exchange . \n        ?exchange ps:P414 wd:Q13677 .\n        ?exchange pq:P249 ?ticker .\n      }\n      OPTIONAL { ?company wdt:P17 ?country. }\n      OPTIONAL { ?company wdt:P159 ?headquarters. }\n      OPTIONAL { ?company wdt:P169 ?ceo. }\n      OPTIONAL { ?company wdt:P452 ?industry. }\n      OPTIONAL { ?company wdt:P199 ?business. }\n      OPTIONAL { ?company wdt:P1056 ?product. }\n#       OPTIONAL { ?company wdt:P169 ?competitor. }\n      OPTIONAL { ?company wdt:P1449 ?nickname. }\n 

In [8]:
print(dow_jones_companies_df)


    SELECT ?company ?companyLabel ?ticker ?headquartersLabel ?ceoLabel ?industryLabel ?businessLabel ?productLabel ?competitorLabel ?nicknameLabel ?subsidiaryLabel ?revenue ?net_income WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
      ?company wdt:P5008 wd:Q165413.
      OPTIONAL { ?company wdt:P249 ?ticker. }
      OPTIONAL { ?company wdt:P159 ?headquarters. }
      OPTIONAL { ?company wdt:P169 ?ceo. }
      OPTIONAL { ?company wdt:P452 ?industry. }
      OPTIONAL { ?company wdt:P1056 ?business. }
      OPTIONAL { ?company wdt:P1057 ?product. }
      OPTIONAL { ?company wdt:P1690 ?competitor. }
      OPTIONAL { ?company wdt:P1449 ?nickname. }
      OPTIONAL { ?company wdt:P3556 ?subsidiary. }
      OPTIONAL { ?company p:P2138/psv:P2138 [wikibase:quantityAmount ?revenue]. }
      OPTIONAL { ?company p:P2139/psv:P2139 [wikibase:quantityAmount ?net_income]. }
    } ORDER BY DESC(?revenue)
    
